# Part I. ETL Pipeline for Pre-Processing the Files

- Student: Jeremy Gu
- Date 2/3/2024




## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [18]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [19]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/businessowner/Documents/GitHub/udacity_aws_data_eng_project1_cassandra_data_modeling


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [20]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [21]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [22]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [23]:
# TO-DO: Create a Keyspace 
try:
	session.execute("""
	CREATE KEYSPACE IF NOT EXISTS udacity
	WITH REPLICATION = 
	{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
	""")
except Exception as e:
	print(e)

#### Set Keyspace

In [24]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
	session.set_keyspace('udacity')
except Exception as e:
	print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [25]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Create table

# Query 1: Session Item History
# Partition Key: sessionId ensures data is distributed across nodes.
# Clustering Column: itemInSession ensures data is sorted within each partition.

query = "CREATE TABLE IF NOT EXISTS session_item_history "
query = query + "(sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))"
try:
	session.execute(query)
except Exception as e:
	print(e)

# Insert data into the table
file = 'event_datafile_new.csv'
with open(file, encoding='utf8') as f:
	csvreader = csv.reader(f)
	next(csvreader)  # skip header
	for line in csvreader:
		query = "INSERT INTO session_item_history (sessionId, itemInSession, artist, song_title, song_length)"
		query = query + " VALUES (%s, %s, %s, %s, %s)"
		try:
			session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
		except Exception as e:
			print(e)

# Select the data
query = "SELECT artist, song_title, song_length FROM session_item_history WHERE sessionId = 338 AND itemInSession = 4"
try:
	rows = session.execute(query)
except Exception as e:
	print(e)

for row in rows:
	print(row.artist, row.song_title, row.song_length)


                    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [26]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item_history (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [27]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

# Select the data to verify it was inserted correctly
query = "SELECT artist, song_title, song_length FROM session_item_history WHERE sessionId = 338 AND itemInSession = 4"
try:
	rows = session.execute(query)
except Exception as e:
	print(e)

for row in rows:
	print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [28]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Create table

# Query 2: User Session History
# Composite Partition Key: (userId, sessionId) ensures data is grouped by user and session.
# Clustering Column: itemInSession ensures data is sorted by item order within each session.



query = "CREATE TABLE IF NOT EXISTS user_session_history "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song_title text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
	session.execute(query)
except Exception as e:
	print(e)

# Insert data into the table
file = 'event_datafile_new.csv'
with open(file, encoding='utf8') as f:
	csvreader = csv.reader(f)
	next(csvreader)  # skip header
	for line in csvreader:
		query = "INSERT INTO user_session_history (userId, sessionId, itemInSession, artist, song_title, firstName, lastName)"
		query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
		try:
			session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
		except Exception as e:
			print(e)

# Select the data
query = "SELECT artist, song_title, firstName, lastName FROM user_session_history WHERE userId = 10 AND sessionId = 182"
try:
	rows = session.execute(query)
except Exception as e:
	print(e)

for row in rows:
	print(row.artist, row.song_title, row.firstname, row.lastname)


                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [29]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Create table

# Query 3: Song Listeners
# Partition Key: song_title ensures data is grouped by song.
# Clustering Column: userId ensures users are uniquely identified within each song group.


query = "CREATE TABLE IF NOT EXISTS song_listeners "
query = query + "(song_title text, userId int, firstName text, lastName text, PRIMARY KEY (song_title, userId))"
try:
	session.execute(query)
except Exception as e:
	print(e)

# Insert data into the table
file = 'event_datafile_new.csv'
with open(file, encoding='utf8') as f:
	csvreader = csv.reader(f)
	next(csvreader)  # skip header
	for line in csvreader:
		query = "INSERT INTO song_listeners (song_title, userId, firstName, lastName)"
		query = query + " VALUES (%s, %s, %s, %s)"
		try:
			session.execute(query, (line[9], int(line[10]), line[1], line[4]))
		except Exception as e:
			print(e)

# Select the data


query = "SELECT firstName, lastName FROM song_listeners WHERE song_title = 'All Hands Against His Own'"
try:
	rows = session.execute(query)
except Exception as e:
	print(e)

for row in rows:
	print(row.firstname, row.lastname)


                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


# Organized Answers

### Query 1: Get Artist, Song Title, and Song Length for Session ID 338, Item in Session 4
- Purpose: Retrieve specific song details based on `sessionId` and `itemInSession`.
- Table Design: `sessionId` as partition key ensures efficient data distribution, and `itemInSession` as clustering column ensures sorting within the partition.

In [31]:
import pandas as pd

# Query 1: Format output with Pandas
query = "SELECT artist, song_title, song_length FROM session_item_history WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows), columns=['Artist', 'Song Title', 'Song Length'])
    print(df)
except Exception as e:
    print(e)

      Artist                       Song Title  Song Length
0  Faithless  Music Matters (Mark Knight Dub)   495.307312


### Query 2: Get Artist, Song, and User Details for UserID 10, SessionID 182
- Purpose: Retrieve all songs played by a specific user in a session, sorted by `itemInSession`.
- Table Design: Composite partition key `(userId, sessionId)` groups data by user and session, while `itemInSession` as clustering column ensures sorting.

In [32]:
# Query 2: Format output with Pandas
query = "SELECT artist, song_title, firstName, lastName FROM user_session_history WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows), columns=['Artist', 'Song Title', 'First Name', 'Last Name'])
    print(df)
except Exception as e:
    print(e)

              Artist                                         Song Title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  First Name Last Name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


### Query 3: Get All Users Who Listened to the Song 'All Hands Against His Own'
- Purpose: Retrieve all users who listened to a specific song.
- Table Design: `song_title` as partition key groups data by song, and `userId` as clustering column ensures unique user identification.

In [33]:
# Query 3: Format output with Pandas
query = "SELECT firstName, lastName FROM song_listeners WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows), columns=['First Name', 'Last Name'])
    print(df)
except Exception as e:
    print(e)

   First Name Last Name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [34]:
## TO-DO: Drop the table before closing out the sessions

# Drop table session_item_history
try:
	session.execute("DROP TABLE IF EXISTS session_item_history")
except Exception as e:
	print(e)

# Drop table user_session_history
try:
	session.execute("DROP TABLE IF EXISTS user_session_history")
except Exception as e:
	print(e)

# Drop table song_listeners
try:
	session.execute("DROP TABLE IF EXISTS song_listeners")
except Exception as e:
	print(e)

### Close the session and cluster connection¶

In [35]:
session.shutdown()
cluster.shutdown()